##### 參考資料：
- https://blog.csdn.net/PIPIXIU/article/details/81016974
    - 建模講解很完整

- https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py
    - 補齊上一篇缺少模型訓練及參數命名錯誤的部分
    

In [1]:
import re
import gc
import string
import pandas as pd
from tqdm import tqdm
from string import digits
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import tensorflow
# Error Solved: Fail to find the dnn implementation
# => https://github.com/tensorflow/tensorflow/issues/24496
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense, Embedding
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import model_from_json
import pickle as pkl
import numpy as np


In [9]:
def clean_eng_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
#     text = re.sub(str([x for x in digits]), " ", text)

    return text

def clean_che_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    text = re.sub(r"[，。、？！：；「」《》·‘•“”?]", "", text)
    text = re.sub(r"\u200b", "", text)
    text = re.sub(str([x for x in digits]), "", text)
    text = re.sub(str([x for x in string.ascii_lowercase]), "", text)

    return text

def preprocess(text_list):
    text_ = [x.lower() for x in text_list]
    text_ = [re.sub("'", '', x) for x in text_]
    return text_

def removePunc(text_list):
    table = str.maketrans('', '', string.punctuation)
    remove_punc_text = []
    for sent in text_list:
        sentence = [w.translate(table) for w in sent.split(' ')]
        remove_punc_text.append(' '.join(sentence))
    return remove_punc_text


#### import dataset

In [10]:
data_path = 'language_data.csv'
df = pd.read_csv(data_path)
df.columns = ['inputs', 'targets']

# df.shape

input_sentences = df.inputs.values.tolist()#[:NUM_SAMPLE]
target_sentences = df.targets.values.tolist()#[:NUM_SAMPLE]


#### cleansing

In [11]:
input_sentences = [clean_eng_text(x) for x in input_sentences]
input_sentences = preprocess(input_sentences)
input_sentences = removePunc(input_sentences)
target_sentences = [clean_che_text(x) for x in target_sentences]
target_sentences = preprocess(target_sentences)
target_sentences = removePunc(target_sentences)

# 句首加'\t'當作起始標誌，句末加'\n'當作終止標誌
target_sentences = ['\t' + x + '\n' for x in target_sentences]

# 確認中英文各自所有的 unique字符
input_ = []
for x in input_sentences:
    for a in x.split(' '):
        input_.append(a)
input_characters = sorted(pd.DataFrame(input_)[0].unique())
target_characters = sorted(list(set(pd.DataFrame(target_sentences)[0].unique().sum())))



#### 生成 LSTM 三維 input
把句子中各字符轉換成 one-hot 編碼，生成LSTM需要的三维输入 `n_samples`, `timestamp`, `one-hot` features

- `NUM_SAMPLES`，样本条数，这里是输入的句子条数
- `INPUT_LENGTH`，输入数据的时刻t的长度，这里为最长的英文句子长度
- `OUTPUT_LENGTH`，输出数据的时刻t的长度，这里为最长的中文句子长度
- `INPUT_FEATURE_LENGTH`，每个时刻进入encoder的lstm单元的数据xtxt的维度，这里为英文中出现的字符数
- `OUTPUT_FEATURE_LENGTH`，每个时刻进入decoder的lstm单元的数据xtxt的维度，这里为中文中出现的字符数


In [12]:
NUM_SAMPLES = int(len(input_sentences))
INUPT_LENGTH = int(max([len(x.split(' ')) for x in input_sentences]))
OUTPUT_LENGTH = int(max([len(x) for x in target_sentences]))
INPUT_FEATURE_LENGTH = int(len(input_characters))
OUTPUT_FEATURE_LENGTH = int(len(target_characters))
print(f'NUM_SAMPLES: {NUM_SAMPLES}, INUPT_LENGTH: {INUPT_LENGTH}, OUTPUT_LENGTH: {OUTPUT_LENGTH}, INPUT_FEATURE_LENGTH: {INPUT_FEATURE_LENGTH}, OUTPUT_FEATURE_LENGTH: {OUTPUT_FEATURE_LENGTH}')

input_dict = {char:index for index, char in enumerate(input_characters)}
input_dict_reverse = {index:char for index, char in enumerate(input_characters)}
target_dict = {char:index for index, char in enumerate(target_characters)}
target_dict_reverse = {index:char for index, char in enumerate(target_characters)}

# encoder输入、decoder输入输出初始化为三维向量
encoder_input = np.zeros((NUM_SAMPLES, INUPT_LENGTH, INPUT_FEATURE_LENGTH), dtype='uint8')
decoder_input = np.zeros((NUM_SAMPLES, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH), dtype='uint8')
decoder_output = np.zeros((NUM_SAMPLES, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH), dtype='uint8')

# 將 input_sentence 進行字符級 one-hot 編碼
for seq_index, seq in enumerate(input_sentences):
    for char_index, char in enumerate(seq.split(' ')):
        encoder_input[seq_index, char_index, input_dict[char]] = 1.0

# 將 target_sentence 進行字符級 one-hot 編碼
for seq_index, seq in enumerate(target_sentences):
    for char_index, char in enumerate(seq):
        decoder_input[seq_index, char_index, target_dict[char]] = 1.0
        if char_index > 0:
            # 训练模型时decoder的输入要比输出晚一个时间步，这样才能对输出监督
            decoder_output[seq_index, char_index-1, target_dict[char]] = 1.0


encoder_input[0]
decoder_input[0]
decoder_output[0]

NUM_SAMPLES: 23444, INUPT_LENGTH: 34, OUTPUT_LENGTH: 44, INPUT_FEATURE_LENGTH: 6626, OUTPUT_FEATURE_LENGTH: 3457


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

#### 構建 encoder-decoder 及 infer 模型

In [13]:
def create_model(n_input, n_output, n_units):
    ###encoder
    encode_input = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    _, encoder_h, encoder_c = encoder(encode_input)
    encoder_state = [encoder_h, encoder_c]

    ###decoder
    decode_input = Input(shape=(None, n_output))
    decoder = LSTM(n_units, return_sequences=True, return_state=True)
    decode_output, _, _ = decoder(decode_input, initial_state=encoder_state)
    decoder_dense = Dense(n_output, activation='softmax')
    decode_output = decoder_dense(decode_output)

    model = Model([encode_input, decode_input], decode_output)
    
    encoder_infer = Model(encode_input, encoder_state)

    decoder_state_input_h = Input(shape=(n_units, ))
    decoder_state_input_c = Input(shape=(n_units, ))
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c]

    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decode_input, initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]
    decoder_infer_output = decoder_dense(decoder_infer_output)
    decoder_infer = Model([decode_input] + decoder_state_input, [decoder_infer_output] + decoder_infer_state)

    return model, encoder_infer, decoder_infer

#### 建構預測 function

In [14]:
def predict_chinese(source, encoder_infer, decoder_infer, n_steps, features):
    # 先推理 encoder，获得预测输入序列的隐状态
    state = encoder_infer.predict(source)
    predict_seq = np.zeros((1, 1, features))
    # 標記起始符處
    predict_seq[0, 0, target_dict['\t']] = 1
    
    output = ''
    for i in range(n_steps):
        yhat, h, c = decoder_infer.predict([predict_seq] + state)
        char_index = np.argmax(yhat[0, -1, :])
        char = target_dict_reverse[char_index]
        output += char  #輸出中文不需空格
        state = [h, c]
        predict_seq = np.zeros((1, 1, features))
        predict_seq[0, 0, char_index] = 1
        if char == '\n':  # 遇到終止符號則停止拚句
            break
    
    return output

#### 模型訓練

In [17]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.


In [18]:
model, encoder_infer, decoder_infer = create_model(n_input=INPUT_FEATURE_LENGTH,
                                                   n_output=OUTPUT_FEATURE_LENGTH,
                                                   n_units=latent_dim)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input, decoder_input], decoder_output, 
          batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save('seq2seq.h5')

    

Train on 18755 samples, validate on 4689 samples
Epoch 1/100
18755/18755 [==============================] - 31s 2ms/sample - loss: 1.0989 - accuracy: 0.0320 - val_loss: 1.9072 - val_accuracy: 0.0295
Epoch 2/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 1.0158 - accuracy: 0.0395 - val_loss: 1.7452 - val_accuracy: 0.0440
Epoch 3/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.9184 - accuracy: 0.0528 - val_loss: 1.6843 - val_accuracy: 0.0535
Epoch 4/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.8472 - accuracy: 0.0613 - val_loss: 1.6621 - val_accuracy: 0.0590
Epoch 5/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.7948 - accuracy: 0.0672 - val_loss: 1.5625 - val_accuracy: 0.0663
Epoch 6/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.7521 - accuracy: 0.0721 - val_loss: 1.5203 - val_accuracy: 0.0723
Epoch 7/100
18755/18755 [==========================

Epoch 55/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1338 - accuracy: 0.1749 - val_loss: 1.7971 - val_accuracy: 0.0861
Epoch 56/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1289 - accuracy: 0.1759 - val_loss: 1.8089 - val_accuracy: 0.0854
Epoch 57/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1249 - accuracy: 0.1768 - val_loss: 1.8092 - val_accuracy: 0.0856
Epoch 58/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1208 - accuracy: 0.1777 - val_loss: 1.8187 - val_accuracy: 0.0846
Epoch 59/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1170 - accuracy: 0.1786 - val_loss: 1.8261 - val_accuracy: 0.0850
Epoch 60/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1132 - accuracy: 0.1794 - val_loss: 1.8302 - val_accuracy: 0.0847
Epoch 61/100
18755/18755 [==============================] - 29s 2ms/sample - loss: 0.1096 - ac

#### 進行英翻中推論預測

In [19]:
for i in range(1000, 1200):
    test = encoder_input[i:i+1, :, :]
    print(test.shape)
    out = predict_chinese(test, encoder_infer, decoder_infer, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH)
    print(input_sentences[i])
    print(out)


(1, 34, 6626)
i like reading
我喜欢阅读

(1, 34, 6626)
i like running
我喜欢跑步

(1, 34, 6626)
i like to read
我喜欢阅读

(1, 34, 6626)
i love my life
我爱我的生活

(1, 34, 6626)
i love my life
我爱我的生活

(1, 34, 6626)
i love my wife
我爱我我的妻子

(1, 34, 6626)
i love parties
我愛派對

(1, 34, 6626)
i met a friend
我遇见一个朋友

(1, 34, 6626)
i must decline
我必须拒绝

(1, 34, 6626)
i must go home
我该回家了

(1, 34, 6626)
i need a raise
我需要加帮大

(1, 34, 6626)
i need a stamp
我需要一張郵票

(1, 34, 6626)
i need it asap
我尽快需要

(1, 34, 6626)
i need my coat
我需要我的大衣

(1, 34, 6626)
i need to know
我需要知道

(1, 34, 6626)
i often hiccup
我经常打嗝

(1, 34, 6626)
i said shut up
我说过了闭嘴

(1, 34, 6626)
i saw five men
我看见五个男人

(1, 34, 6626)
i saw five men
我看见五个男人

(1, 34, 6626)
i saw her swim
我看見她游泳

(1, 34, 6626)
i should do it
我该做什么

(1, 34, 6626)
i study korean
我学来了

(1, 34, 6626)
i want to play
我想去玩

(1, 34, 6626)
i was a doctor
我以前是醫生

(1, 34, 6626)
i was learning
我在學習

(1, 34, 6626)
i was not drunk
我沒醉

(1, 34, 6626)
i will sue you
我要告你

(1, 34, 6626)
i 